# 层级索引

当目前为止，我们接触的都是一维数据和二维数据，用 `Pandas` 的 `Series` 和 `DataFrame` 对象就可以存储。  
但我们也经常会遇到存储多维数据的需求，数据索引超过一两个键。因此，`Pandas` 提供了 `Panel` 和 `Panel4D` 对象解决三维数据与四维数据。  
而在实践中，更直观的形式是通过**层级索引**（`hierarchical indexing`，也被称为**多级索引**，`multi-indexing`）配合多个有不同**等级**（`level`）的一级索引一起使用，  
这样就可以将高维数组转换成类似一维 `Series` 和二维 `DataFrame` 对象的形式。  

在这一节中，我们将介绍创建 `MultiIndex` 对象的方法，多级索引数据的取值、切片和统计值的计算，以及普通索引与层级索引的转换方法。

In [1]:
import numpy as np
import pandas as pd

## 1. 多级索引Series

让我们看看如何用一维的 `Series` 对象表示二维数据——用一系列包含特征与数值的数据点来简单演示。

### 1.1. 笨方法

In [2]:
index = [('California', 2000), ('California', 2010), ('New York', 2000), ('New York', 2010), ('Texas', 2000), ('Texas', 2010)]
populations = [33871648, 37253956, 18976457, 19378102, 20851820, 25145561]
pop = pd.Series(populations, index=index)  # 通过元组构成的多级索引
pop

(California, 2000)    33871648
(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
(Texas, 2010)         25145561
dtype: int64

In [3]:
pop[('California', 2010):('Texas', 2000)]

(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
dtype: int64

In [4]:
pop[[i for i in pop.index if i[1] == 2010]]  # 假如你想要选择所有 2000 年的数据... ...

(California, 2010)    37253956
(New York, 2010)      19378102
(Texas, 2010)         25145561
dtype: int64

### 1.2. 好方法：Pandas多级索引

`Pandas` 提供了更好的解决方案。用元组表示索引其实是多级索引的基础，`Pandas` 的 `MultiIndex` 类型提供了更丰富的操作方法。我们可以用元组创建一个多级索引。

In [5]:
index = pd.MultiIndex.from_tuples(index)
index

MultiIndex([('California', 2000),
            ('California', 2010),
            (  'New York', 2000),
            (  'New York', 2010),
            (     'Texas', 2000),
            (     'Texas', 2010)],
           )

In [6]:
pop = pop.reindex(index)
pop

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [7]:
pop[:, 2010]  # 假如你想要选择所有 2000 年的数据 ^v^

California    37253956
New York      19378102
Texas         25145561
dtype: int64

### 1.3. 高级数据的多级索引

你可能已经注意到，我们其实完全可以用一个带行列索引的简单 `DataFrame` 代替前面的多级索引。  
其实 `Pandas` 已经实现了类似的功能。`unstack()` 方法可以快速将一个多级索引的 `Series` 转化为普通索引的 `DataFrame`：

In [8]:
pop_df = pop.unstack()  # unstack() 方法“升维”数据
pop_df

,2000,2010
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


In [9]:
pop_df.stack()          # stack() 方法实现相反的效果，“降维”数据

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

你可能会纠结于为什么要费时间研究层级索引。其实理由很简单：  
如果我们可以用含多级索引的一维 `Series` 数据表示二维数据，那么我们就可以用 `Series` 或 `DataFrame` 表示三维甚至更高维度的数据。  
多级索引每增加一级，就表示数据增加一维，利用这一特点就可以轻松表示任意维度的数据了。

In [10]:
# 给DataFrame“升维”
pop_df = pd.DataFrame({'total': pop,
                       'under18': [9267089, 9284094, 4687374, 4318033, 5906301, 6879014]})
pop_df

total  under18
California 2000  33871648  9267089
           2010  37253956  9284094
New York   2000  18976457  4687374
           2010  19378102  4318033
Texas      2000  20851820  5906301
           2010  25145561  6879014

In [11]:
f_u18 = pop_df['under18'] / pop_df['total']  # 前面介绍过的通用函数和其他功能也同样适用于层级索引。
f_u18

California  2000    0.273594
            2010    0.249211
New York    2000    0.247010
            2010    0.222831
Texas       2000    0.283251
            2010    0.273568
dtype: float64

In [12]:
f_u18.unstack()

,2000,2010
California,0.273594,0.249211
New York,0.247010,0.222831
Texas,0.283251,0.273568


## 2. 多级索引的创建方法

为 `Series` 或 `DataFrame` 创建多级索引最直接的办法就是将 `index` 参数设置为至少二维的索引数组。

In [13]:
df = pd.DataFrame(np.random.rand(4, 2),
                  index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                  columns=['data1', 'data2'])
df  # MultiIndex 的创建工作将在后台完成

data1     data2
a 1  0.520543  0.492040
  2  0.272888  0.378713
b 1  0.650696  0.202062
  2  0.288338  0.871132

In [14]:
data = {('California', 2000): 33871648,
        ('California', 2010): 37253956,
        ('Texas', 2000): 20851820,
        ('Texas', 2010): 25145561,
        ('New York', 2000): 18976457,
        ('New York', 2010): 19378102}
pd.Series(data)  # 如果你把将元组作为键的字典传递给 Pandas，Pandas 也会默认转换为 MultiIndex

California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
New York    2000    18976457
            2010    19378102
dtype: int64

### 2.1. 显式地创建多级索引

你可以用 `pd.MultiIndex` 中的类方法更加灵活地构建多级索引。

In [15]:
pd.MultiIndex.from_arrays([['a', 'a', 'b', 'b'], [1, 2, 1, 2]])  # 通过一个有不同等级的若干简单数组组成的列表来构建 MultiIndex

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

In [16]:
pd.MultiIndex.from_tuples([('a', 1), ('a', 2), ('b', 1), ('b', 2)])  # 通过包含多个索引值的元组构成的列表创建 MultiIndex

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

In [17]:
pd.MultiIndex.from_product([['a', 'b'], [1, 2]])  # 用两个索引的笛卡尔积（Cartesian product）创建 MultiIndex

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

In [18]:
# pd.MultiIndex(levels=[['a', 'b'], [1, 2]],
#               labels=[[0, 0, 1, 1], [0, 1, 0, 1]])  # 直接提供 levels（包含每个等级的索引值列表的列表）和 labels（包含每个索引值标签列表的列表）创建 MultiIndex

pd.MultiIndex(levels=[['a', 'b'], [1, 2]],
              codes=[[0, 0, 1, 1], [0, 1, 0, 1]])    # 从0.24.0版本开始，MultiIndex.labels已经改名为MultiIndex.codes

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

### 2.2. 多级索引的等级名称

给 `MultiIndex` 的等级加上名称会为一些操作提供便利。  
你可以在前面任何一个 `MultiIndex` 构造器中通过 `names` 参数设置等级名称，也可以在创建之后通过索引的 `names` 属性来修改名称。

In [19]:
pop.index.names = ['state', 'year']
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

### 2.3. 多级列索引

每个 `DataFrame` 的行与列都是对称的，也就是说既然有多级行索引，那么同样可以有多级列索引。

In [20]:
# 多级行列索引
index = pd.MultiIndex.from_product([[2013, 2014], [1, 2]], names=['year', 'visit'])
columns = pd.MultiIndex.from_product([['Bob', 'Guido', 'Sue'], ['HR', 'Temp']], names=['subject', 'type'])

# 模拟数据
data = np.round(np.random.randn(4, 6), 1)
data[:, ::2] *= 10
data += 37

# 创建 DataFrame
health_data = pd.DataFrame(data, index=index, columns=columns)  # 这相当于一份简易的四维数据
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      47.0  38.7  29.0  36.5  39.0  36.5
     2      57.0  36.6  39.0  37.1  43.0  36.1
2014 1      49.0  36.5  41.0  38.5  25.0  37.0
     2      44.0  37.1  54.0  36.2  20.0  36.0

In [21]:
health_data['Guido']

type          HR  Temp
year visit            
2013 1      29.0  36.5
     2      39.0  37.1
2014 1      41.0  38.5
     2      54.0  36.2

## 3. 多级索引的取值与切片

对 `MultiIndex` 的取值和切片操作很直观，你可以直接把索引看成额外增加的维度。

### 3.1. Series多级索引

In [22]:
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [23]:
pop['California', 2000]

33871648

`MultiIndex` 也支持**局部取值**（`partial indexing`），即只取索引的某一个层级。  

In [24]:
pop['California']  # 假如只取最高级的索引，获得的结果是一个新的 Series，未被选中的低层索引值会被保留。

year
2000    33871648
2010    37253956
dtype: int64

In [25]:
pop.loc['California':'New York']  # 类似的还有局部切片，不过要求 `MultiIndex` 是按顺序排列的。

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
dtype: int64

In [26]:
pop[:, 2000]  # 如果索引已经排序，那么可以用较低层级的索引取值，第一层级的索引可以用空切片

state
California    33871648
New York      18976457
Texas         20851820
dtype: int64

In [27]:
pop[pop > 22000000]  # 其他取值与数据选择的方法也都起作用，例如通过布尔掩码选择数据

state       year
California  2000    33871648
            2010    37253956
Texas       2010    25145561
dtype: int64

In [28]:
pop[['California', 'Texas']]  # 用花哨的索引选择数据

state       year
California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
dtype: int64

### 3.2. DataFrame多级索引

`DataFrame` 多级索引的用法与 `Series` 类似。

In [29]:
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      47.0  38.7  29.0  36.5  39.0  36.5
     2      57.0  36.6  39.0  37.1  43.0  36.1
2014 1      49.0  36.5  41.0  38.5  25.0  37.0
     2      44.0  37.1  54.0  36.2  20.0  36.0

In [30]:
health_data['Guido', 'HR']  # 由于 DataFrame 的基本索引是列索引，因此 Series 中多级索引的用法到了 DataFrame 中就应用在列上了。

year  visit
2013  1        29.0
      2        39.0
2014  1        41.0
      2        54.0
Name: (Guido, HR), dtype: float64

In [31]:
health_data.iloc[:2, :2]    # 与单索引类似，前面介绍的 loc、iloc 和 ix 索引器都可以使用

subject      Bob      
type          HR  Temp
year visit            
2013 1      47.0  38.7
     2      57.0  36.6

In [32]:
health_data.loc[:, ('Bob', 'HR')]  # 虽然这些索引器将多维数据当作二维数据处理，但是在 loc 和 iloc 中可以传递多个层级的索引元组

year  visit
2013  1        47.0
      2        57.0
2014  1        49.0
      2        44.0
Name: (Bob, HR), dtype: float64

In [33]:
health_data.loc[(:, 1), (:, 'HR')]  # 这种索引元组的用法不是很方便，如果在元组中使用切片还会导致语法错误

SyntaxError: invalid syntax (4211402498.py, line 1)

In [34]:
idx = pd.IndexSlice
health_data.loc[idx[:,1], idx[:,'HR']]  # 虽然你可以用 Python 内置的 slice() 函数获取想要的切片，但是还有一种更好的办法，就是使用 IndexSlice 对象。Pandas 专门用它解决这类问题

,subject,Bob,Guido,Sue
,type,HR,HR,HR
year,visit,,,
2013,1,47.0,29.0,39.0
2014,1,49.0,41.0,25.0


## 4. 多级索引行列转换

使用多级索引的关键是掌握有效数据转换的方法。`Pandas` 提供了许多操作，可以让数据在内容保持不变的同时，按照需要进行行列转换。  
之前我们用一个简短的例子演示过 `stack()` 和 `unstack()` 的用法，但其实还有许多合理控制层级行列索引的方法，让我们来一探究竟。

### 4.1. 有序的索引和无序的索引

如果 `MultiIndex` 不是有序的索引，那么大多数切片操作都会失败。

In [35]:
index = pd.MultiIndex.from_product([['a', 'c', 'b'], [1, 2]])
data = pd.Series(np.random.rand(6), index=index)
data.index.names = ['char', 'int']
data  # 创建一个不按字典顺序（lexographically）排列的多级索引 Series

char  int
a     1      0.170518
      2      0.723187
c     1      0.273586
      2      0.988780
b     1      0.584792
      2      0.681615
dtype: float64

In [36]:
# 如果想对索引使用局部切片，那么错误就会出现
try:
    data['a':'b']
except KeyError as e:
    print(type(e))
    print(e)

<class 'pandas.errors.UnsortedIndexError'>
'Key length (1) was greater than MultiIndex lexsort depth (0)'


局部切片和许多其他相似的操作都要求 `MultiIndex` 的各级索引是有序的（即按照字典顺序由 `A` 至 `Z`）。  
为此，`Pandas` 提供了许多便捷的操作完成排序，如 `sort_index()` 和 `sortlevel()` 方法。

In [37]:
data = data.sort_index()
data

char  int
a     1      0.170518
      2      0.723187
b     1      0.584792
      2      0.681615
c     1      0.273586
      2      0.988780
dtype: float64

In [38]:
data['a':'b']

char  int
a     1      0.170518
      2      0.723187
b     1      0.584792
      2      0.681615
dtype: float64

### 4.2. 索引stack与unstack

我们可以将一个多级索引数据集转换成简单的二维形式，可以通过 `level` 参数设置转换的索引层级。

In [39]:
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [40]:
pop.unstack(level=0)

state,California,New York,Texas
year,,,
2000,33871648,18976457,20851820
2010,37253956,19378102,25145561


In [41]:
pop.unstack(level=1)

year,2000,2010
state,,
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


In [42]:
pop.unstack().stack()  # unstack() 是 stack() 的逆操作，同时使用这两种方法让数据保持不变

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

### 4.3. 索引的设置与重置

层级数据维度转换的另一种方法是行列标签转换，可以通过 `reset_index` 方法实现。

In [43]:
pop_flat = pop.reset_index(name='population')
pop_flat

,state,year,population
0,California,2000,33871648
1,California,2010,37253956
2,New York,2000,18976457
3,New York,2010,19378102
4,Texas,2000,20851820
5,Texas,2010,25145561


In [44]:
pop_flat.set_index(['state', 'year'])

population
state      year            
California 2000    33871648
           2010    37253956
New York   2000    18976457
           2010    19378102
Texas      2000    20851820
           2010    25145561

## 5. 多级索引的数据累计方法

对于层级索引数据，可以设置参数 `level` 实现对数据子集的累计操作。

In [45]:
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      47.0  38.7  29.0  36.5  39.0  36.5
     2      57.0  36.6  39.0  37.1  43.0  36.1
2014 1      49.0  36.5  41.0  38.5  25.0  37.0
     2      44.0  37.1  54.0  36.2  20.0  36.0

In [46]:
# data_mean = health_data.mean(level='year')  # 这种写法将被遗弃，这种语法其实就是 GroupBy 功能的快捷方式
data_mean = health_data.groupby(level='year').mean()
data_mean

subject   Bob        Guido          Sue      
type       HR   Temp    HR   Temp    HR  Temp
year                                         
2013     52.0  37.65  34.0  36.80  41.0  36.3
2014     46.5  36.80  47.5  37.35  22.5  36.5

In [47]:
# data_mean.mean(axis=1, level='type')  # 这种写法将被遗弃，这种语法其实就是 GroupBy 功能的快捷方式
data_mean.groupby(axis=1, level='type').mean()

type,HR,Temp
year,,
2013,42.333333,36.916667
2014,38.833333,36.883333


## 6. Panel 数据

这里还有一些 `Pandas` 的基本数据结构没有介绍到，包括 `pd.Panel` 对象和 `pd.Panel4D` 对象。这两种数据结构可以分别看成是（一维数组）`Series` 和（二维数组）`DataFrame` 的三维与四维形式。  
如果你熟悉 `Series` 和 `DataFrame` 的使用方法，那么 `Panel` 和 `Panel4D` 使用起来也会很简单，`ix`、`loc` 和 `iloc` 索引器在高维数据结构上的用法更是完全相同。  
但是本书并不打算进一步介绍这两种数据结构，我个人认为多级索引在大多数情况下都是更实用、更直观的高维数据形式。  
另外，`Panel` 采用密集数据存储形式，而多级索引采用稀疏数据存储形式。在解决许多真实的数据集时，随着维度的不断增加，密集数据存储形式的效率将越来越低。  
但是这类数据结构对一些有特殊需求的应用还是有用的。如果你想对 `Panel` 与 `Panel4D` 数据结构有更多的认识，请参见 3.14 节。